# Building a classifier for Spam/Non-Spam Emails

**DATASET**
- A Spambase dataset from the UCI Machine Learning Repository: http://archive.ics.uci.edu/ml/datasets/Spambase
- The Spambase data set consists of 4,601 e-mails, of which 1,813 are spam (39.4%). The data set archive contains a processed version of the e-mails wherein 57 real-valued features have been extracted and the spam/non-spam label has been assigned. The data set archive contains a description of the features extracted as well as some simple statistics over those features.

**AIM**
- Build a classification model which will be able to filter spam/not spam emails. 
- Compare the performance of a few classification algorithms and choose the best performing one. 
- Evaluate the models using k-fold cross-validation. 
- Reduce the false postives i.e marking good mail as spam.

**EVALUATION/OUTPUT**
- A table with one row per fold showing the false positive, false negative, and overall error rates, and one final row corresponding to the average error rates across all folds for our best performing model.

In [1]:
# STEP 1: Importing all the required libraries.
import pandas as pd
import numpy as np
import urllib.request
import sklearn.ensemble as ske
from sklearn import tree, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

**The following class stores all the relevant details regarding algorithms. It contains the following computed details in a np-array for all the kfolds:**
- True Negative
- True Positive
- False Negative
- False Positive
- Accuracy across K-Folds
- Error rates 

In [2]:
#STEP 2: Creating a class to store the output metrics for all the folds algorithm wise. 
class AlgorithmOutputs:
    
    def __init__(self, algo, tn, fp, fn, tp):
        self.algo = algo
        self.tn = tn
        self.fp = fp
        self.fn = fn
        self.tp = tp
        self.total_accuracy = self.compute_accuracy()

    def compute_error_rates(self):
        error_rate = (self.fp + self.fn)/(self.tn + self.fp + self.tp + self.fn)
        return error_rate
    
    def compute_accuracy(self):
        error_rates = self.compute_error_rates()
        return 1 - np.squeeze(np.sum(error_rates))/error_rates.shape[0]

### Dataset
- As stated above, we are using the spambase dataset from the UCI Machine Learning Repository - http://archive.ics.uci.edu/ml/datasets/Spambase
- For reading the data we are using _read_csv_ function provided by pandas. It reads a data file into a DataFrame.
- Data preprocessing is an important step in ML as the quality of data and the useful information that can be derived from it directly affects the ability of our model to learn. Therefore, it is extremely important that we preprocess our data before feeding it into our model. 
- Irregular distribution of values may suffer from poor performance during learning and sensitivity to input values resulting in higher error. 
- One can understand how varied the data is from 'Attribute Statistics' in https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.DOCUMENTATION
- For normalization of data, we are going to use MinMaxScaler. 

**We can always start by fitting our model to raw, normalized and standardized data and compare the performance for best results.**

For the above given dataset the results were more or less the same. But feature scaling is a good technique when the machine learning algorithms are sensitive to feature scaling such as gradient decsent which tends to converge faster to the local maximum/minimum if the features are scaled properly. However, there are few ML algorithms such as tree based algoithms which are virtually invariant to it.

In [3]:
# STEP 3: Preparing Dataset
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data')
with urllib.request.urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.names') as response:
    data_col = response.read()
    data_col = data_col.decode('utf-8')
# Extracting column names from spambase.names
data_col = data_col.split('|')
data_col = data_col[len(data_col)-1].splitlines()
data_col = data_col[2:]
data_col_details =  {}
for colDetails in data_col:
    colDetails = colDetails.split(':')
    data_col_details[colDetails[0]] = ''.join(e for e in colDetails[1] if e.isalnum())
data_col_details['Spam'] = 'nominal'
data_col_names = data_col_details.keys()
X = data.iloc[:, :-1]
y = data.iloc[:,-1:]

# STEP 4: Preprocessing the data before allowing the models to learn from our dataset.
# Transform features by scaling each feature to a given range.
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

Since, our aim is to classify emails as Spam or Non-Spam emails, it is important to make false-positive (classifying a Non-Spam email as a Spam email) as minimum as possible. To handle this, we will make some changes in how we will predict output. We will try to change the cutoff probabilities to reduce the number of false positives. While converting predicted output to probabilities, **we will instruct the model to classify only those with probability more than 0.9 to be marked as spam.** It can decrease the overall accuracy, but it's better to read some non-important email in the inbox, than leaving out some important messages in the spam. 

Default cutoff's to classify an event are 0.5. Increasing or decreasing this threshold might affest your accuarcy.

In [4]:
#Step 5: Creating Custom Scorers as per the output required.
def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]
def accuracy(y_true, y_pred): return accuracy_score(y_true, (y_pred > 0.5)) # To reduce the number of false positives
scoring = {'tp': make_scorer(tp), 'tn': make_scorer(tn),
           'fp': make_scorer(fp), 'fn': make_scorer(fn), 'accuracy': make_scorer(accuracy)}

### Models

Once, we are done with processing the data and loading it into appropriate data structures, let's analyse which classification algorithms will perform better. In the end, the accuracy score and confusion matrix will tell us how well our model works.

**Some of the models which can be used for the given dataset and the classification problem are as follows:** 

- **DecisionTree**: A faster way to give accurate results on large data set.

- **Random Forest**: A tree based algorithm which is designed to handle missing values and mantain accuracy for a large data size. It works well with cross validation. One can think of Random Forest Algorithm a way of combining multple Decision Tree together. RFs train each tree independently, using a random sample of the data.

- **Gradient Boosting**: It is similar to Random Forest but GBT build trees one at a time, where each new tree helps to correct errors made by previously trained tree. 

- **Logistic Regression**: It measures the relationship between the categorical dependent variable and one or more independent variables by calculating probabilities using a logistic function.

- **MultinomialNB**: Naive Bayes is very popular probabilistic classifier which is based on the assumption that features are independent to each other. There are three types of Naive Bayes model under the scikit-learn library:
    1. **Gaussian:** It is used in classification and it assumes that features follow a normal distribution.
    2. **Multinomial:** It is used for discrete counts. 
    3. **Bernoulli:** It is useful if your feature vectors are binary (i.e. zeros and ones)
    
_Please note for all the listed models we have made an educated estimation for the parameters. Parameter tunings has been done to give us the best results._

Hyperparameter tuning relies on experimental results than theory, there doesn't exist any predefined set of values which will sure shot give you the best results. Therefore, if we have to find the best parameters for our data and problem, trial and error is the best plausible option. We can determine the optimal settings by trying multiple combinations and evaluate the performance of each model. However, evaluating each model only on the training set can lead to overfitting. 

**Cross Validation (CV) is the technique to overcome overfitting.** We split our data into a training and a testing set. In K-Fold CV, we split our training data into K subsets, called folds. We then iteratively fit the model K times, each time training the data on K-1 of the folds and evaluating on the Kth fold (called the validation data). To elucidate, consider fitting a model with K = 10. In the first iteration, we train our model on the first nine folds and evaluate on the tenth. We repeat this procedure 9 more times, each time evaluating on a different fold. At the very end of training, we average the performance on each of the folds to come up with final validation metrics for the model.

In [5]:
#Step 6: Selecting various classification algorithms/models for comparison. 
# Hyperparameter tuning done using the calculated guess. Algorithms and their parameters are selected depending 
# upon our requirement and dataset knowledge. 
algorithms = {
        "DecisionTree": tree.DecisionTreeClassifier(max_depth=10),
        "RandomForest": ske.RandomForestClassifier(n_estimators=50),
        "GradientBoosting": ske.GradientBoostingClassifier(n_estimators=50),
        "MultinomialNB": MultinomialNB(),
        "BernoulliNB": BernoulliNB(),
        "LogisticRegression": LogisticRegression(solver='liblinear', penalty='l1')
}
results = []
print("\nNow testing algorithms")

#Step 7: Performing K-Folds CV(K=10) on our models. 
for algo in algorithms:
    clf = algorithms[algo]
    # scikit-learn allows us to perform K-Folds CV using the below function. 
    # It evaluate metrics by cross-validation. 
    cv_results = cross_validate(clf, X, y.values.ravel(), cv=10, scoring=scoring) 
    out = AlgorithmOutputs(algo, cv_results['test_tn'], cv_results['test_fp'], 
                                            cv_results['test_fn'], cv_results['test_tp'])
    print("Accuracy of %s is %0.2f" % (algo, out.total_accuracy * 100) + "%")
    results.append(out)


Now testing algorithms
Accuracy of DecisionTree is 90.93%
Accuracy of RandomForest is 93.96%
Accuracy of GradientBoosting is 93.33%
Accuracy of MultinomialNB is 87.54%
Accuracy of BernoulliNB is 88.50%
Accuracy of LogisticRegression is 91.07%


**Random Forest** gives the maximum accuracy. It's accuracy is ~94%. Also, RF ensures the **minimum false positive rate.**

In [6]:
#Step 8: Determining the most accurate model which gives you the least possible false positive.
def compute_max(p):
    return p.total_accuracy
winner = max(results, key=compute_max) # to determine the model which gives us the maximum accuracy.
print("Winner Algorithm is %s with error rate of %0.2f" %(winner.algo, (1 - winner.total_accuracy)*100) + "%.")

Winner Algorithm is RandomForest with error rate of 6.04%.


### Output
We will show the output in a form of a table (DataFrame) with one row per fold showing the false positive, false negative, and overall error rates, and add one final row corresponding to the average error rates across all folds for our best performing model.

In [7]:
#Step 9: For the winner algorithm, generating results folds wise. Also, showing the final/overall error rate. 
output_data = {'Fold' : np.arange(1, 11, 1).tolist(), 'False Positive' : winner.fp, 
               'False Negative' : winner.fn, 'Error Rates' : winner.compute_error_rates()}
df = pd.DataFrame(data=output_data)
df = df.append(pd.Series(df.mean()), ignore_index=True)
df.iloc[-1, df.columns.get_loc('Fold')] = 'Overall/Average'
df

,Fold,False Positive,False Negative,Error Rates
0,1,5.0,20.0,0.054348
1,2,9.0,16.0,0.054348
2,3,6.0,23.0,0.063043
3,4,5.0,17.0,0.047826
4,5,5.0,12.0,0.036957
5,6,11.0,9.0,0.043478
6,7,4.0,9.0,0.028261
7,8,4.0,8.0,0.026087
8,9,35.0,13.0,0.104348
9,10,34.0,33.0,0.145652


#### RESULT
We have successfully created and implemented classification model using multiple algorithms with parameter tuning. We have evaluated the models using kfold validation. We identified that in our case the **random forest** algorithm worked better than all the algorithms with the error rate ~6%.